## Lab 2 – Datarensning (ETL) för Smart Home IoT – “dirty version”
# Mål med labben

I denna labb ska du:

* Läsa in ett smutsigt IoT-dataset för

* smarta hem.

* Undersöka datakvalitet:

* saknade värden

* dubbletter

* orimliga värden (outliers)

* Rensa och standardisera datan    (Transform i ETL).

Slutligen Spara en ren version av datan för Lab 3 & 4 (Load).

Steg 0 – Importera bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Steg 1 – Läs in smutsig data

För att läsa in och spara datafiler, finns det två sätt via Git eller Google driver. Förlj länken nedan där har jag förklarat hur man gör beroende på vilken väg ni väljer
https://www.notion.so/Lab2-Datarensning-2aa9de1f0d8380ab9e64e60eb26c7656?source=copy_link

In [ ]:
# läsa in den smutsiga datan, sparade från lab 1

In [ ]:
# Visa de första raderna
df.head()

Steg 2 – Överblick av datasetet

In [ ]:
df.info()

df.describe()

Steg 3 – Kontrollera datakvalitet

4.1 Saknade värden

In [ ]:
print("Saknade värden per kolumn")
display(df.isna().sum())  # isna().sum() räknar hur många NaN finns det i varje kolumn

4.2 Dubbletter

In [ ]:
print("Antal dubbletter (hela rader som är kopior):", df.duplicated().sum())
dups = df[df.duplicated()]


In [ ]:
dups['Home ID'].value_counts() # om man inte vilka ex home id som är duplicated

In [ ]:
dups[dups['Home ID'] == 94]


4.3 Orimliga värden (business rules)

In [ ]:
# kollar först outliers för energyforbrukning
energy_outliers = df[(df['Energy Consumption (kWh)'] < 0) |
                     (df['Energy Consumption (kWh)'] > 1000)]
print("Antal orimliga energivärden:", len(energy_outliers))
energy_ouliners

In [ ]:
# kollar outliers för temp
temp_outliers = df[(df['Outdoor Temperature (°C)'] < -50) |
                   (df['Outdoor Temperature (°C)'] > 60)]
print("Antal orimliga temperaturer:", len(temp_outliers))
temp_outliners

Steg 5 – Visualisera problemet (outlier-spik)

In [ ]:
# visualisera 500 första raderna för att visualisera outliers
df['Energy Consumption (kWh)'].head(500).plot(kind='line', title='Energiförbrukning över tid')
plt.ylabel('kWh')
plt.show()

In [ ]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.sort_values('DateTime', inplace=True)

df.head(500).plot(x='DateTime', y='Energy Consumption (kWh)', marker='o')
plt.title('Energiförbrukning över tid (med outlier)')
plt.ylabel('kWh')
plt.show()

Steg 6 – Rensa datan

6.1 – Hantera saknade värden, och NaN

* Energy Consumption (kWh)

* Outdoor Temperature (°C)

6.1.1 Energy Consumption (kWh)

In [ ]:
# skapar en kopia av datan
df_cleaned = df.copy()

In [ ]:
print(f"Före rensning: {len(df_cleaned)} rader")

# 1. Beräkna medelvärden (skippar automatiskt NaN värden)
energy_mean = df_cleaned['Energy Consumption (kWh)'].mean()

# 2. Fylla ALLA saknade värden (NaN, tomma celler) med medelvärden
missing_energy_before = df_cleaned['Energy Consumption (kWh)'].isnull().sum()
df_cleaned['Energy Consumption (kWh)'].fillna(energy_mean, inplace=True)

print(f"Fyllde {missing_energy_before} energi-NaN med medelvärde: {energy_mean:.2f} kWh")

6.1.2 Outdoor Temperature (°C)

In [ ]:
print(f"Före rensning: {len(df_cleaned)} rader")

# 1. Beräkna medelvärden (skippar automatiskt NaN värden)
temp_mean = df_cleaned['Outdoor Temperature (°C)'].mean()

# 2. Fylla ALLA saknade värden (NaN, tomma celler) med medelvärden
missing_temp_before = df_cleaned['Outdoor Temperature (°C)'].isnull().sum()

df_cleaned['Outdoor Temperature (°C)'].fillna(temp_mean, inplace=True)

print(f"Fyllde {missing_temp_before} temperatur-NaN med medelvärde: {temp_mean:.1f}°C")

In [ ]:
print("Saknade värden efter rensning:")
display(df.isna().sum())

6.2 Hantera outliers

6.2.1 Outlier Energy Consumption

In [ ]:
#  Ersätt outliers och felaktiga värden med medelvärden

# Ersätt negativ energi och extremvärden (>15 kWh) med medelvärde
energy_outliers = (df_cleaned['Energy Consumption (kWh)'] < 0) | (df_cleaned['Energy Consumption (kWh)'] > 15)
outlier_count_energy = energy_outliers.sum()
df_cleaned.loc[energy_outliers, 'Energy Consumption (kWh)'] = energy_mean
print(f"Ersatte {outlier_count_energy} energi-outliers med medelvärde")

6.2.2 Oulier Outdoor Temperature

In [ ]:
# Ersätt extrema temperaturer (<-40 eller >50) med medelvärde
temp_outliers = (df_cleaned['Outdoor Temperature (°C)'] < -40) | (df_cleaned['Outdoor Temperature (°C)'] > 50)
outlier_count_temp = temp_outliers.sum()
df_cleaned.loc[temp_outliers, 'Outdoor Temperature (°C)'] = temp_mean
print(f"Ersatte {outlier_count_temp} temperatur-outliers med medelvärde")

print(f"Alla rader behållna: {len(df_cleaned)} rader")

6.3 Ta bort alla dubbletter

In [ ]:
# 2. Ta bort dubbletter
df_cleaned = df_cleaned.drop_duplicates()
print(f"Efter dubbletter: {len(df_cleaned)} rader")

Steg 7 – Hitta felaktiga textvärden innan rensning

7.1 Hitta unika värden i textkolumnerna

In [ ]:
print("Unika värden i Appliance Type:")
display(df_clean['Appliance Type'].unique())

print("\nUnika värden i Season:")
display(df_clean['Season'].unique())

7.2 Hitta värden som innehåller mellanslag före/efter

In [ ]:
mask_spaces = df_clean['Appliance Type'].str.contains(r'^\s|\s$', regex=True)
df_clean[mask_spaces][['Appliance Type', 'Home ID', 'Date', 'Time']]

7.3 Hitta värden med blandning av stora och små bokstäver

In [ ]:
mask_case = df_clean['Appliance Type'].str.match(r'[A-Za-z ]+') & ~df['Appliance Type'].str.istitle()
df_clean[mask_case][['Appliance Type', 'Home ID']]


Steg 8– Standardisera text och datatyper

In [ ]:
# standardisera texfält- ta bort mellanslag, rättar case
df['Appliance Type'] = df['Appliance Type'].str.strip().str.title()
df['Season'] = df['Season'].str.strip().str.capitalize()


In [ ]:
# säkerställer att datum är date time format
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

In [ ]:
#vi kan validera att datan har rensat
dups = df[df.duplicated(keep=False)]
dups['Home ID'].value_counts()

Steg 9 - Validera rensad data

In [ ]:
# Är rensningen OK?
print(f"Före: {len(df_original)} rader → Efter: {len(df_cleaned)} rader")
print(f" Saknade värden: {df_cleaned.isnull().sum().sum()}")
print(f" Dubbletter: {df_cleaned.duplicated().sum()}")
print(f" Negativ energi: {(df_cleaned['Energy Consumption (kWh)'] < 0).sum()}")

df_cleaned.head()

Steg 10 – Spara ren version

In [ ]:

# spara rensat data till csv file
df_clean.to_csv(
    '/content/drive/MyDrive/big_data_labs/cleaned_iot_data.csv',
    index=False
)

print("Ren version sparad som cleaned_iot_data.csv")

Steg 11 –  Beräkna Rolling Mean & Diff på ren data

In [ ]:
# Sortera i tidsordning
df = df.sort_values('DateTime')

# Rullande medelvärde
df['RollingMean'] = df['Energy Consumption (kWh)'].rolling(window=50).mean()

# Differens mellan mätpunkter
df['Diff'] = df['Energy Consumption (kWh)'].diff()

In [ ]:
# visa grafen efter rensning
plt.figure(figsize=(12,6))

plt.plot(df['DateTime'].head(500),
         df['Energy Consumption (kWh)'].head(500),
         label='Rådata', alpha=0.3)

plt.plot(df['DateTime'].head(500),
         df['RollingMean'].head(500),
         label='Rullande medelvärde', linewidth=2)

plt.xlabel('Tid')
plt.ylabel('kWh')
plt.title('Energiförbrukning – efter datarensning')
plt.legend()

# Fix för x-axeln:
ax = plt.gca()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))  # Max 10 etiketter
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
